# Automated CAD-to-CFD Workflow: Internal Pipe Flow

**Author:** Mohammad Afshar
**Date:** January 2026
**Tools:** CadQuery (Python 3.13.11), OpenFOAM v13, ParaView, vs code, pc with wsl

## 1. Project Overview
This notebook demonstrates an automated engineering workflow for fluid dynamics. Instead of manually drawing geometry in a GUI, I use **Python** to parametrically generate a fluid domain. This geometry is then automatically processed and exported for Computational Fluid Dynamics (CFD) analysis in OpenFOAM.

### Key Objectives:
1.  **Parametric Design:** Generate a pipe geometry where diameter and length can be changed via variables.
2.  **Automated Selection:** Programmatically identify and separate `Inlet`, `Outlet`, and `Wall` boundaries.
3.  **Physics Verification:** Simulate Laminar Flow and verify the fully developed velocity profile.

## 2. Theoretical Background
To ensure a stable Laminar flow simulation, we calculate the Reynolds Number ($Re$) to size our pipe and velocity correctly.

$$Re = \frac{\rho v D}{\mu}$$

For this simulation, we target a Reynolds number below the turbulent threshold ($Re < 2300$).
* **Fluid:** Water ($\nu \approx 1 \times 10^{-6} m^2/s$)
* **Diameter:** $25.4 mm$

$$Re = \frac{v \times 0.0254}{10^{-6}} = 2000$$

* **Target Velocity:** $0.078 m/s$

At this low velocity, we expect a parabolic velocity profile to develop fully within the $1000 mm$ pipe length.

### Step 1: Initialize OpenFOAM Environment
We start by copying the standard `motorBike` tutorial. This provides us with the necessary `snappyHexMesh` configuration files, which we will later modify for our pipe.

* **Source:** `/opt/openfoam13/.../motorBike`
* **Destination:** `$FOAM_RUN/pipeProject`

In [ ]:
%%bash
cp -r /opt/openfoam13/tutorials/incompressibleFluid/motorBike/motorBike $FOAM_RUN/pipeProject
cd $FOAM_RUN/pipeProject

### Step 2: Parametric Geometry Generation
We use **CadQuery** to generate the fluid domain programmatically.

Instead of exporting the entire object as one STL, we identify and export the boundaries (`inlet`, `outlet`, `walls`) separately. This allows OpenFOAM to apply different physics conditions (e.g., *Velocity Inlet* vs *No-Slip Wall*) to each surface automatically.

* **Fluid Domain:** Cylinder ($L=1000mm, d=25.4mm$)
* **Export Path:** `./constant/geometry/` (Standard for OpenFOAM v13)

### importing related lib

In [ ]:
import os
import cadquery as cq

### Creating the geometry

for simplicity I am doing each operation line by line.

creating some variables that we can refer to them later

In [ ]:
diameter = 0.0254
length = 1

In [ ]:
#select a workplane on a plane
fluid_domain = cq.Workplane("YZ")
#sketch a circle on that workplane
fluid_domain= fluid_domain.circle(diameter)
#extrude the circle along the X axis
fluid_domain= fluid_domain.extrude(length)

at this stage you can visualize your model in vs code by using the following

In [ ]:
from ocp_vscode import show, set_port
set_port(3939)
show(fluid_domain)

### 2. Select the Boundaries
We identify faces based on their position relative to the axes.

In [ ]:
#INLET: The face at the "bottom" of the extrude (negative X)
inlet_face = fluid_domain.faces("<X")
# OUTLET: The face at the "top" of the extrude (positive X)
outlet_face = fluid_domain.faces(">X")
# WALLS: The rounded face parallel to the X axis
# "|X" selector means "faces parallel to X axis"
# and not |x mean all faces except those parallel to X axis because we have a cylinder
wall_face = fluid_domain.faces("not |X")


### 3. Export separate STL files
OpenFOAM will use these filenames as the patch names if you load them individually

just  some simple variables so the directories of the files or somewhat automated

In [ ]:
rawinletpath = "$FOAM_RUN/pipeProject/constant/geometry/inlet.stl"
rawoutletpath = "$FOAM_RUN/pipeProject/constant/geometry/outlet.stl"
rawwallpath = "$FOAM_RUN/pipeProject/constant/geometry/walls.stl"
# 2. Expand the variable to the actual system path
fullinletpath = os.path.expandvars(rawinletpath)
fulloutletpath = os.path.expandvars(rawoutletpath)
fullwallpath = os.path.expandvars(rawwallpath)

exporting the faces to stl files to the related directories

In [ ]:
inlet_face.export(fullinletpath)
outlet_face.export(fulloutletpath)
wall_face.export(fullwallpath)

Openfoam

%%bash

cd $FOAM_RUN/pipeProject

is just that we are in the correct directory because the jupyter notebook launches bash in the home directory and not in the specific folder that we need.

cat << 'EOF' > system/blockMeshDict

and this is so that we can write some text and rewrite it to a specific document 

OpenFOAM's `snappyHexMesh` is a "cut-cell" mesher. It does not generate a mesh from nothing; instead, it requires a background grid which it then carves into the shape of our geometry.

We modify `system/blockMeshDict` to create a bounding box slightly larger than our pipe.
We add a **1mm buffer** on all sides to ensuring the fluid domain is fully captured within the meshable area.

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
cat << 'EOF' > system/blockMeshDict
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2206                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      blockMeshDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

scale   1;

vertices
(
    (-0.1 -0.03 -0.03)
    (1.1 -0.03 -0.03)
    (1.1  0.03 -0.03)
    (-0.01  0.03 -0.03)
    (-0.01 -0.03  0.03)
    (1.1 -0.03  0.03)
    (1.1  0.03  0.03)
    (-0.01  0.03  0.03)
);

blocks
(
    hex (0 1 2 3 4 5 6 7) (20 10 10) simpleGrading (1 1 1)
);

edges
(
);

boundary
(
    bounding
    {
        type patch;
        faces
        (
            (0 1 5 4)
            (1 2 6 5)
            (2 3 7 6)
            (3 0 4 7)
            (0 3 2 1)
            (4 5 6 7)
        );
    }
);

// ************************************************************************* //
EOF

This dictionary controls how the mesh is carved.

1.  **Geometry Import:** We linked our three STL files (`inlet`, `outlet`, `walls`) and assigned them specific names.
    * *Note:* OpenFOAM v13 requires the explicit `file` keyword.
2.  **Refinement Surfaces:** We told the mesher to refine near these surfaces to capture the curvature of the pipe accurately.
3.  **Location In Mesh:** We set the coordinate `(0.5 0 0)` as the "seed point."
    * **important:** This point must be **inside** the fluid. `snappyHexMesh` keeps the region containing this point and deletes everything else.

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
cat << 'EOF' > system/snappyHexMeshDict
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2206                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      snappyHexMeshDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

castellatedMesh true;
snap            true;
addLayers       false;

geometry
{
    inlet.stl
    {
        type triSurfaceMesh;
        file "inlet.stl";      // Explicit file definition required in v13
        name inlet;
    }
    outlet.stl
    {
        type triSurfaceMesh;
        file "outlet.stl";
        name outlet;
    }
    walls.stl
    {
        type triSurfaceMesh;
        file "walls.stl";
        name walls;
    }
}

castellatedMeshControls
{
    maxLocalCells 100000;
    maxGlobalCells 2000000;
    minRefinementCells 100;
    maxLoadUnbalance 0.10;
    nCellsBetweenLevels 3;

    features
    (
    );

    refinementSurfaces
    {
        inlet
        {
            level (2 2);
            patchInfo { type patch; }
        }
        outlet
        {
            level (2 2);
            patchInfo { type patch; }
        }
        walls
        {
            level (2 2);
            patchInfo { type wall; }
        }
    }

    resolveFeatureAngle 30;
    locationInMesh (0.5 0 0); 
    allowFreeStandingZoneFaces true;
}

snapControls
{
    nSmoothPatch 3;
    tolerance 2.0;
    nSolveIter 30;
    nRelaxIter 5;
    nFeatureSnapIter 10;
    implicitFeatureSnap false;
    explicitFeatureSnap true;
    multiRegionFeatureSnap false;
}

addLayersControls
{
    relativeSizes true;
    layers
    {
        walls
        {
            nSurfaceLayers 3;
        }
    }
    expansionRatio 1.0;
    finalLayerThickness 0.3;
    minThickness 0.1;
    nGrow 0;
    featureAngle 60;
    slipFeatureAngle 30;
    nRelaxIter 3;
    nSmoothSurfaceNormals 1;
    nSmoothNormals 3;
    nSmoothThickness 10;
    maxFaceThicknessRatio 0.5;
    maxThicknessToMedialRatio 0.3;
    minMedianAxisAngle 90;
    nBufferCellsNoExtrude 0;
    nLayerIter 50;
}

meshQualityControls
{
    maxNonOrtho 65;
    maxBoundarySkewness 20;
    maxInternalSkewness 4;
    maxConcave 80;
    minVol 1e-13;
    minTetQuality 1e-30;
    minArea -1;
    minTwist 0.02;
    minDeterminant 0.001;
    minFaceWeight 0.02;
    minVolRatio 0.01;
    minTriangleTwist -1;
    nSmoothScale 4;
    errorReduction 0.75;
}

mergeTolerance 1e-6;

// ************************************************************************* //
EOF


### Mesh Controls
we tweak the mesh settings in two specific files.

#### 1. Global Resolution (`system/blockMeshDict`)
* Look for: `blocks ( hex ... (20 10 10) ... )`
(x y z) number of cells
increase/decrease the numbers

#### 2. Local Refinement (`system/snappyHexMeshDict`)

(min max) refinement layers
* Look for: `level (2 2);` inside `refinementSurfaces`.
* **The Logic:**
    * **Level 0:** Base size (same as blockMesh).
    * **Level 1:** Split cell in half (2x detail).
    * **Level 2:** Split cell again (4x detail).
* **Tweak:** Change `(2 2)` to `(3 3)` for very smooth curved walls, or `(1 1)` for a rough, fast check.

#### 3. Boundary Layers (Advanced)
* Look for: `addLayers false;` at the top of `snappyHexMeshDict`.
* **Tweak:** Set to `true` to insert thin "prism layers" along the walls. This is critical for accurate friction calculations in Turbulent flow, but optional for this simple Laminar case.

run the meshing tools

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
blockMesh
snappyHexMesh -overwrite

Since `simpleFoam` is a **Steady-State** solver, the "Time" here actually represents **Iterations**.

**Key Settings we defined:**
1.  **`application simpleFoam;`**: Tells OpenFOAM which solver to use.
2.  **`endTime 500;`**: The simulation will try to solve the equations 500 times.
3.  **`writeInterval 50;`**: Instead of saving only the final result, we save the data every 50 iterations (0, 50, 100...).
    * *Why?* This allows us to animate the convergence in ParaView to see how the flow develops from a "guess" to the final solution.

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
cat << 'EOF' > system/controlDict
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v13                                   |
|   \\  /    A nd           | Website:  openfoam.org                          |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      controlDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

application     simpleFoam;

startFrom       startTime;

startTime       0;

stopAt          endTime;

endTime         500;    // Run for 500 iterations

deltaT          1;      // Each step is 1 iteration

writeControl    timeStep;

writeInterval   50;     // <--- SAVE DATA EVERY 50 STEPS (Change this if needed)

purgeWrite      0;      // 0 = Keep all files. (Set to 5 to keep only last 5)

writeFormat     ascii;

writePrecision  6;

writeCompression off;

timeFormat      general;

timePrecision   6;

runTimeModifiable true;

// ************************************************************************* //
EOF

just defining that we want our fluid to have laminar properties.

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
cat << 'EOF' > constant/turbulenceProperties
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v13                                   |
|   \\  /    A nd           | Website:  openfoam.org                          |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      turbulenceProperties;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

simulationType  laminar;

// ************************************************************************* //
EOF


In the `constant/transportProperties` file, we define the physical properties of the fluid.

Since we are running an incompressible simulation (`simpleFoam`), the solver uses **Kinematic Viscosity** ($\nu$) instead of dynamic viscosity.

**We configured this for Water at 20°C:**
**Viscosity ($\nu$):** $1 \times 10^{-6} m^2/s$
    * *Comparison:* Air would be $\approx 1.5 \times 10^{-5}$ (15x more viscous kinematically).

**Understanding the Units:**
The line `nu [0 2 -1 0 0 0 0]` represents the SI dimensions:
**Length:** 2 ($m^2$)
**Time:** -1 ($s^{-1}$)
* Result: $m^2/s$

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
cat << 'EOF' > constant/transportProperties
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v13                                   |
|   \\  /    A nd           | Website:  openfoam.org                          |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      transportProperties;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

transportModel  Newtonian;

nu              [0 2 -1 0 0 0 0] 1e-06;

// ************************************************************************* //
EOF

the 0 directory contains the initial conditions. The file U specifically tells the solver how the fluid is momving at the start of the simulation and how it behaves at the boundaries.


We define the flow behavior in the `0/U` file. This is where we apply our physics calculations.

**Boundary Configuration:**
1.  **Inlet (`fixedValue`):** We force water into the pipe at a constant speed of **0.078 m/s**.
    * *Why this speed?* As calculated in the "Theoretical Background," this ensures a Reynolds number of ~250, allowing the flow to fully develop within our 100mm pipe.
2.  **Outlet (`zeroGradient`):** We tell the solver "Let the flow leave the pipe without obstruction." The velocity at the outlet is calculated based on what is happening inside.
3.  **Walls (`noSlip`):** This is the fundamental rule of fluid mechanics. The fluid touching the wall has **zero velocity** due to friction.

**Internal Field:**
* We start the simulation assuming the water inside is completely still (`0 0 0`). The solver will then "push" the inlet water in over time.

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
rm 0/k 0/omega 0/nut 0/nuTilda 2>/dev/null
cat << 'EOF' > 0/U
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v13                                   |
|   \\  /    A nd           | Website:  openfoam.org                          |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       volVectorField;
    object      U;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

dimensions      [0 1 -1 0 0 0 0];

internalField   uniform (0 0 0);

boundaryField
{
    // Fluid enters at 0.078 m/s in X-direction
    inlet
    {
        type            fixedValue;
        value           uniform (0.078 0 0);
    }

    // Gradient is zero at outlet (let it flow out)
    outlet
    {
        type            zeroGradient;
    }

    // No slip at walls (friction)
    walls
    {
        type            noSlip;
    }
}

// ************************************************************************* //
EOF


We define the pressure field in the `0/p` file.

**Important Note on Units:**
Since `simpleFoam` is an incompressible solver, it calculates **Kinematic Pressure** ($P / \rho$).
* **Unit:** $m^2/s^2$ (instead of Pascals).
* To get Pascals later, simply multiply the result by the fluid density (e.g., $1000 kg/m^3$ for water).

**Boundary Configuration:**
1.  **Outlet (`fixedValue` 0):** We set the outlet to **0** (Gauge Pressure). This gives the solver a reference point.
2.  **Inlet (`zeroGradient`):** We do **not** set the pressure here. We let the solver calculate how much pressure is required to push the fluid at the requested 0.078 m/s. This resulting value is our **Pressure Drop ($\Delta P$)**.
3.  **Walls (`zeroGradient`):** Pressure does not have a "no-slip" condition. It assumes the pressure gradient normal to the wall is zero.

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
cat << 'EOF' > 0/p
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v13                                   |
|   \\  /    A nd           | Website:  openfoam.org                          |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    object      p;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

dimensions      [0 2 -2 0 0 0 0];

internalField   uniform 0;

boundaryField
{
    inlet
    {
        type            zeroGradient;
    }

    outlet
    {
        type            fixedValue;
        value           uniform 0;
    }

    walls
    {
        type            zeroGradient;
    }
}

// ************************************************************************* //
EOF


The file `0/nuTilda` represents the **Modified Turbulent Viscosity**, which is specific to the Spalart-Allmaras turbulence model.

Even though we are running a **Laminar** simulation (where turbulence = 0), we initialize this file to keep our case "Turbulence-Ready."
* If we later enable turbulence, this file provides the starting values.
* **Inlet:** Set to `0` (Clean flow, no turbulence entering).
* **Walls:** Set to `0` (Turbulence dies at the wall).

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
# 1. Fix nuTilda (The one that crashed)
cat << 'EOF' > 0/nuTilda
FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    object      nuTilda;
}
dimensions      [0 2 -1 0 0 0 0];
internalField   uniform 0;
boundaryField
{
    inlet   { type fixedValue; value uniform 0; }
    outlet  { type zeroGradient; }
    walls   { type zeroGradient; }
}
EOF

The file `0/k` represents the energy contained in the turbulent eddies of the fluid.

**Why define this for Laminar flow?**
Similar to `nuTilda`, we include it so If we switch to a turbulence model later, OpenFOAM will look for this file.

* **Physical Meaning:** $k = \frac{3}{2}(U I)^2$, where $I$ is turbulence intensity.
* **Inlet:** We set a small non-zero value (`0.24`) to represent minute background disturbances, though in a purely Laminar solver, this is ignored.
* **Walls:** We use a `kqRWallFunction` which is a smart boundary condition. It acts like a "Zero Value" condition but is numerically stable for complex meshes.

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
# 2. Fix k (Turbulent Kinetic Energy)
cat << 'EOF' > 0/k
FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    object      k;
}
dimensions      [0 2 -2 0 0 0 0];
internalField   uniform 0;
boundaryField
{
    inlet   { type fixedValue; value uniform 0; }
    outlet  { type zeroGradient; }
    walls   { type zeroGradient; }
}
EOF

The file `0/omega` defines the frequency of turbulence dissipation ($1/s$).
current simulation is Laminar, including `omega` makes this case compatible with the **$k-\omega$ SST** model. This is the "Gold Standard" turbulence model for engineering because it handles walls much better than $k-\epsilon$.

**Inlet:** Set to a small background value (`0.24`).
**Walls (`omegaWallFunction`):** This is critical.

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
# 3. Fix omega (Specific Dissipation Rate)
cat << 'EOF' > 0/omega
FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    object      omega;
}
dimensions      [0 0 -1 0 0 0 0];
internalField   uniform 0;
boundaryField
{
    inlet   { type fixedValue; value uniform 0; }
    outlet  { type zeroGradient; }
    walls   { type zeroGradient; }
}
EOF

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
# 4. Fix epsilon (Dissipation Rate)
cat << 'EOF' > 0/epsilon
FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    object      epsilon;
}
dimensions      [0 2 -3 0 0 0 0];
internalField   uniform 0;
boundaryField
{
    inlet   { type fixedValue; value uniform 0; }
    outlet  { type zeroGradient; }
    walls   { type zeroGradient; }
}
EOF

The file `0/epsilon` defines the rate at which turbulent energy decays ($m^2/s^3$).
This ensures compatibility with the standard **$k-\epsilon$** turbulence model, which is the most common model.

* **$k$ (Energy):** The size/strength of the eddies.
* **$\epsilon$ (Dissipation):** How fast those eddies break down and disappear.
* **Relationship:** High $\epsilon$ means turbulence dies out quickly.

**Boundary Configuration:**
1.  **Inlet:** Set to a small background value.
2.  **Walls (`epsilonWallFunction`):** Similar to `omega`, we use a wall function to approximate the complex behavior of dissipation near the wall without needing an impossibly fine mesh.

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
# 5. Fix nut (Calculated is actually okay for this one, but let's reset it to be safe)
cat << 'EOF' > 0/nut
FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    object      nut;
}
dimensions      [0 2 -1 0 0 0 0];
internalField   uniform 0;
boundaryField
{
    inlet   { type calculated; value uniform 0; }
    outlet  { type calculated; value uniform 0; }
    walls   { type calculated; value uniform 0; }
}
EOF

execute

In [ ]:
%%bash
cd $FOAM_RUN/pipeProject
simpleFoam
paraFoam